### Dependencies

In [1]:
import numpy as np # Scientific computing 
import pandas as pd # Dataframe to organize content
import os # Creating folders
import pickle # Serializing module
import FATS # Feature extraction library
import sqlite3 # Database library
import urllib # Reading and saving webpages

/home/amanjot_bhullar/anaconda3/envs/py27/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


### Extract Features from all Star Data
A detailed explanation of the features that are extracted is given on http://isadoranun.github.io/tsfeat/FeaturesDocumentation.html.  In the below code, features are extracted from each data-set and stored in a database.

In [6]:
# A list of all star classes.
stars = os.listdir('Time_Series/')

# Initializing database and cursor.
star_data_db = sqlite3.connect('star_data.db')
star_data_cursor = star_data_db.cursor()
    
# Initializing table--(star type) and data type--(BLOB).
star_data_cursor.execute("CREATE TABLE IF NOT EXISTS star(star_name BLOB, star_features BLOB)")


for star in stars:
    print(star)
    # Read Data.
    star_data_path = 'Time_Series/'+star
    star_data = pd.read_csv(star_data_path, sep="\s+", names=["time", "magnitude", "error"])

    # Creating lists from pandas dataframe (these are the parameters used to compute the features).
    magnitude = star_data.magnitude.tolist()
    time = star_data.time.tolist()
    error = star_data.error.tolist()

    #Extracting Features.
    parameters = np.array([magnitude, time, error])
    feature_space = FATS.FeatureSpace(Data=['magnitude','time', 'error'])
    features = feature_space.calculateFeature(parameters)
    features_array = features.result(method='array')

    # Convert to float so that python3 can decode it.
    temp = [float(x) for x in features_array]

    # Serializing features so that it can be stored in database.
    features_pickled = pickle.dumps(temp, pickle.HIGHEST_PROTOCOL)

    # Storing star_data in database for future reference.
    star_data_cursor.execute("INSERT INTO star(star_name, star_features) VALUES (?,?)",
                             (star, sqlite3.Binary(features_pickled)))
    star_data_db.commit()


print("Data stored.")

['test1.dat', 'test2.dat', 'test3_t2cep.dat']
test1.dat


/home/amanjot_bhullar/anaconda3/envs/py27/lib/python2.7/site-packages/scipy/optimize/_minimize.py:394: RuntimeWarning: Method nelder-mead cannot handle constraints nor bounds.
  RuntimeWarning)


test2.dat
test3_t2cep.dat
Data stored.
